# 한국어 언어모델 학습 및 다중 과제 튜닝
## KoGPT-2 기반의 챗봇 실습

> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)

In [1]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 79 not upgraded.
Need to get 6,229 kB of archives.
After this operation, 14.5 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 2.13.3 [6,229 kB]
Fetched 6,229 kB in 1s (6,513 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 160987 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.13.3_a

In [2]:
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Git LFS initialized.
Cloning into 'kogpt2'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 43 (delta 16), reused 0 (delta 0)
Unpacking objects: 100% (43/43), done.


In [3]:
!pip install transformers

     |████████████████████████████████| 2.1MB 7.9MB/s 
     |████████████████████████████████| 901kB 37.5MB/s 
     |████████████████████████████████| 3.3MB 38.3MB/s 


In [4]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [5]:
tokenized_text = tokenizer.encode('이순신은 조선 중기의 무신이다.', add_special_tokens=True)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['▁이순', '신은', '▁조선', '▁중기의', '▁무신', '이다', '.']
[10925, 6647, 1117, 40249, 39793, 128, 47440]


In [6]:
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode("이순신은", add_special_tokens=True).ids).unsqueeze(0).to('cuda')

output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=100, num_return_sequences=3)
for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print("GENERATED SEQUENCE : {0}".format(tokenizer.decode(generated_sequence, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GENERATED SEQUENCE : 이순신은 백건(장옹진 분)을 찾아가 "형 오늘 집에 없다.</s><s> 이 대통령은 또 “우리 기업들이 지금 어려운 때인데 수출로 위기를 넘기지만은 더 큰 고민이 있다”면서 “세계 경제가 다시 활기를 띠고 있다는 것을 전 세계가 주목하고 있다”고 언급했다.</s><s> 이어 “우리도 ‘글로벌녹색성장연구소’(GGI)를 만드는 노력을 해오고 있고 유엔 인권이사회(UNH)를 유치하는 등 국제사회에 대한 기여를 하고 있다”면서 “자국 기업들이
GENERATED SEQUENCE : 이순신은 “김갑석, 이항진, 이순재 다 알고 있다”면서 “이홍곤은 내가 잘 아는 사람”이라 거들었다.</s><s> 또한, 고엽제 전몰장병들은 지난 60년간 미국인들에게 ‘비참한 삶’을 체험하게 했다.</s><s> 그러나 이런 전쟁 때문에 참전 전·여군에게 고통·참등감이 큰 경우가 많다.</s><s> 특히 고엽제 전우회 등은 “고엽제 피해자들은 전쟁 때문에 전쟁 속에서 죽을 수 있는데도
GENERATED SEQUENCE : 이순신은 “사천에 내려준 곳은 천하에 없는 곳이 하나도 없을 것 같아 마음이 답답하다”고 말하며 눈물을 흘렸다.</s><s> 이는 “대선에 대한 국민의 알권리가 충분히 보호받을 수 있도록 하는 것이 정부의 의무”라는 박근혜 대통령의 발언과 맥락을 같이하는 부분이다.</s><s> 이는 ‘국민들의 알권리를 충족시킬 수 있는 수단’이라는 박 대통령의 발언의 취지가 반영된 것이다.</s><s> 더욱이 이 발언은 박 대통령이 최근 각 방송사로부터 입수한 ‘국가정보원 해킹 등


## 데이터셋 준비

In [7]:
!git clone https://github.com/songys/Chatbot_data.git

Cloning into 'Chatbot_data'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 42 (delta 14), reused 0 (delta 0), pack-reused 18
Unpacking objects: 100% (42/42), done.


In [10]:
import pandas as pd
data = pd.read_csv('/content/Chatbot_data/ChatbotData .csv')

In [11]:
data.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [12]:
added_special_token_num = tokenizer.add_special_tokens(['<s>', '</s>'])
print(added_special_token_num)

0


In [13]:
pad_id = tokenizer.token_to_id("<pad>")
print(pad_id)
tokenizer.enable_padding(pad_id=pad_id, pad_token="<pad>")
tokenizer.enable_truncation(max_length=128)

3


In [14]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, file_path):
        self.data = []
        self.file_path = file_path
        self.tokenizer = tokenizer
    
    def load_data(self):
        raw_data = pd.read_csv(self.file_path)
        train_data = '<s>'+raw_data['Q']+'</s>'+'<s>'+raw_data['A']+'</s>'
        #<s>안녕하세요</s><s> -> 네, 안녕하세요</s>
        tokenized_train_data = tokenizer.encode_batch(train_data)
        for single_data in tokenized_train_data:
            self.data.append(torch.tensor(single_data.ids).unsqueeze(0))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data[index]
        return item

In [16]:
train_dataset = ChatDataset(tokenizer=tokenizer, file_path='/content/Chatbot_data/ChatbotData .csv')
train_dataset.load_data()

In [17]:
from torch.utils.data import DataLoader

data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [18]:
from transformers import AdamW

In [19]:
optimizer = AdamW(model.parameters(), lr=1e-4, correct_bias=True)

epochs = 3

avg_loss = (0.0, 0.0)
for epoch in range(epochs):
    count=0
    for data in data_loader:
        optimizer.zero_grad()
        data = data.transpose(1,0)
        data = data.to('cuda')
        model = model.to('cuda')
        
        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss = loss.to('cuda')
        loss.backward()
        avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
        optimizer.step()
        if count % 200 == 0:
            print('epoch no.{0}  train ({1}/{2})  loss = {3:.5f}  avg_loss = {4:.5f}' . format(epoch, count, len(data_loader), loss, avg_loss[0] / avg_loss[1]))
        count += 1


epoch no.0  train (0/2956)  loss = 2.78418  avg_loss = 2.78418
epoch no.0  train (200/2956)  loss = 1.12382  avg_loss = 1.42267
epoch no.0  train (400/2956)  loss = 1.03242  avg_loss = 1.29560
epoch no.0  train (600/2956)  loss = 0.98358  avg_loss = 1.22743
epoch no.0  train (800/2956)  loss = 0.95088  avg_loss = 1.23343
epoch no.0  train (1000/2956)  loss = 1.41702  avg_loss = 1.17254
epoch no.0  train (1200/2956)  loss = 1.08221  avg_loss = 1.18450
epoch no.0  train (1400/2956)  loss = 1.34610  avg_loss = 1.18205
epoch no.0  train (1600/2956)  loss = 1.28548  avg_loss = 1.15730
epoch no.0  train (1800/2956)  loss = 0.91892  avg_loss = 1.14943
epoch no.0  train (2000/2956)  loss = 1.40398  avg_loss = 1.15482
epoch no.0  train (2200/2956)  loss = 0.89574  avg_loss = 1.11031
epoch no.0  train (2400/2956)  loss = 1.05893  avg_loss = 1.14295
epoch no.0  train (2600/2956)  loss = 1.27120  avg_loss = 1.12967
epoch no.0  train (2800/2956)  loss = 1.02778  avg_loss = 1.09695
epoch no.1  train

In [20]:
torch.save(model.state_dict(), 'chitchat_model.bin')

In [21]:
def encoding(text):
    text = '<s>'+text+'</s><s>'
    return torch.tensor(tokenizer.encode(text).ids).unsqueeze(0).to('cuda')

def decoding(ids):
    return tokenizer.decode_batch(ids)

tokenizer.no_padding()
tokenizer.no_truncation()

e_s = tokenizer.token_to_id('</s>')
unk = tokenizer.token_to_id('<unk>')

In [22]:
def get_answer(input_sent):
    input_ids = encoding(input_sent)

    sample_outputs = model.generate(
        input_ids,
        num_return_sequences=5,
        do_sample=True, 
        max_length=128, 
        top_k=50, 
        top_p=0.95, 
        eos_token_id=e_s,
        early_stopping=True,
        bad_words_ids=[[unk]]
    )

    decoded_result = decoding(sample_outputs.tolist())
    for result in decoded_result:
        print(result)

In [23]:
get_answer('안녕?')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


안녕? 안녕하세요.
안녕? 안녕하세요.
안녕? 안녕하세요.
안녕? 안녕하세요.
안녕? 안녕.


In [24]:
get_answer('만나서 반가워.')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


만나서 반가워. 좋은 분이었나봐요.
만나서 반가워. 좋은 시간 보내시길 바라요.
만나서 반가워. 좋은 만남이었길 바라요.
만나서 반가워. 고마워요.
만나서 반가워. 좋은 만남이었길 바라요.


In [25]:
get_answer('인공지능의 미래에 대해 어떻게 생각하세요?')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


인공지능의 미래에 대해 어떻게 생각하세요? 무엇이 인공지능을 탄생시켰을까요. 인공지능은 살면서 인간보다 인공지능을 더 똑똑해지려 하고 있어요. 인공지능은 인간보다 인공지능을 더 편하게 만들어주지 않지요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능이 만드는 수많은 기기가 만나는 실시간 현상입니다.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능은 이미 수십 년 전에 이미 등장해서 인지할 수 있어요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능이 그런 일을 계속 하시게 할 수는 없을 거예요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능이 만들어질거 같아요. 인공지능을 대체할 인공지능이 나타날 거예요. 인공지능은 인공지능연구소를 설립할거예요.


In [26]:
get_answer('여자친구 선물 추천해줘.')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


여자친구 선물 추천해줘. 같이 여행을 떠나보는 것도 좋을 것 같아요.
여자친구 선물 추천해줘. 좋은 선물이 될 것 같아요.
여자친구 선물 추천해줘. 직접 물어보는 게 좋을 것 같아요.
여자친구 선물 추천해줘. 마음에 드는 걸 선물해보세요.
여자친구 선물 추천해줘. 그녀가 꼭 좋아하는 것만은 아니랍니다.


In [27]:
get_answer('앞으로 인공지능이 어떻게 발전하게 될까요?')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


앞으로 인공지능이 어떻게 발전하게 될까요? 인공지능이 뭘 원하는지 알아보세요. 인공지능은 더 똑똑해질 거예요.
앞으로 인공지능이 어떻게 발전하게 될까요? 인공지능은 점점 더 똑똑해질 거예요. 인공지능이 사람들을 분류하고 있나요.
앞으로 인공지능이 어떻게 발전하게 될까요? 인공지능은 정말 방대한 양의 데이터를 만들어낼 수 있어요. 인공지능이 정말 무궁무진하니까요. 인공지능이 만들어낸 미래에는 수도 없이 넘실거리고 있을지도 몰라요.
앞으로 인공지능이 어떻게 발전하게 될까요? 인공지능은 초미세먼지도 줄일 수 있어요.
앞으로 인공지능이 어떻게 발전하게 될까요? 인공지능은 앞으로 어떻게 더 똑똑해 질까요?


In [28]:
get_answer('이제 그만 수업 끝내자.')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


이제 그만 수업 끝내자. 선생님께 말씀해보세요.
이제 그만 수업 끝내자. 공부하기 좋은 환경은 아니에요.
이제 그만 수업 끝내자. 후회하지 않기 위한 선택일 거라 생각해요.
이제 그만 수업 끝내자. 하고 싶은 걸 말씀해보세요.
이제 그만 수업 끝내자. 마음이 복잡하겠어요.
